<a href="https://colab.research.google.com/github/Mario-RJunior/web-scraping-ML/blob/master/scraping_Mercado_Livre.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importando as bibliotecas
from bs4 import BeautifulSoup
import requests
import pandas as pd
from time import sleep
import re
import numpy as np

In [2]:
re_precos = '<span class="price__fraction">(.*)</span>'
re_quartos = r'\| (.*) quarto'
re_areas = '> (.*) m² '
re_quarto_area_ausente = "> (.*) quarto"

In [3]:
zonas = ['norte', 'sul', 'leste', 'oeste']
paginas = ['', '_Desde_49', '_Desde_97', '_Desde_145', '_Desde_193', 
           '_Desde_241', '_Desde_289']
dados = {'zona': [],
         'quartos': [],
         'area': [],
         'preco': []}

for zona in zonas:
  for pagina in paginas:
    url = f'https://imoveis.mercadolivre.com.br/apartamentos/aluguel/sao-paulo/sao-paulo-zona-{zona}/{pagina}'

    c = requests.get(url).content
    soup = BeautifulSoup(c, 'html.parser')

    preco = soup.find_all(name='span', attrs={'class': 'price__fraction'})
    atributos = soup.find_all(name='div', attrs={'class': 'item__attrs'})

    for p in preco:
      valor = re.findall(re_precos, str(p))[0]
      dados['preco'].append(valor)
    
    for atributo in atributos:
      n_quartos = re.findall(re_quartos, str(atributo))
      area = re.findall(re_areas, str(atributo))

      if n_quartos == []:
        n_quartos = re.findall(re_quarto_area_ausente, str(atributo))
        if n_quartos == []:
          n_quartos = np.nan
        else:
          n_quartos = n_quartos[0]
      else:
        n_quartos = n_quartos[0]
      dados['quartos'].append(n_quartos)

      if area == []:
        area = np.nan
      else:
        area = area[0]      
      dados['area'].append(area)

      dados['zona'].append(zona)
    sleep(5)

In [4]:
# Gerando do DataFrame
df = pd.DataFrame(dados)

In [5]:
# Visualizando o DataFrame
df

,zona,quartos,area,preco
0,norte,2,65,2.500
1,norte,3,65,1.500
2,norte,2,50,1.550
3,norte,2,35,1.100
4,norte,1,59,1.250
...,...,...,...,...
1051,oeste,3,900,15.000
1052,oeste,3,170,5.000
1053,oeste,2,73,5.000
1054,oeste,2,98,4.160


***Importante***:

Note na coluna "preco" que os dados da escala de milhar apresentam pontos, o que representa um problema uma vez que o Python irá interpretar esses valores como sendo um float. Por exemplo, um aluguel igual a 6 mil reais será considerado como 6 reais fazendo com que nossa escala fique completamente distorcida, gerando problemas que posteriormente irão afetar os algorítmos. Devemos então tratar tais valores.

In [6]:
# Alterando os dados da coluna preco para remover os pontos
df['preco'] = df['preco'].map(lambda x: x.replace('.', ''))

In [7]:
df.head(10)

,zona,quartos,area,preco
0,norte,2,65,2500
1,norte,3,65,1500
2,norte,2,50,1550
3,norte,2,35,1100
4,norte,1,59,1250
5,norte,2,50,1200
6,norte,1,29,1099
7,norte,1,30,900
8,norte,1,30,900
9,norte,1,30,1250


In [8]:
# Salvando o DataFrame em um arquivo .csv
df.to_csv('dados_imoveis.csv')